In [26]:
# libraries
import pandas as pd
import os
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.callbacks import get_openai_callback
from tqdm.auto import tqdm

In [51]:
%pip install openai == 0.28
%pip install tiktoken
%pip install tqdm
%pip install matplotlib
%pip install sympy
import numpy as np
import openai
import tiktoken
from tqdm.auto import trange, tqdm
import time
import os
import json
from tqdm import tqdm
import re
from types import NoneType
import multiprocessing.dummy
from io import StringIO
from contextlib import redirect_stdout
import signal
from contextlib import contextmanager
import matplotlib.pyplot as plt
import sys
import ast
import copy
import pandas as pd
from IPython.display import display_latex
from langchain.callbacks import get_openai_callback
from tqdm.auto import tqdm

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '=='


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [52]:
%pip install --upgrade --quiet  langchain-google-genai pillow

Note: you may need to restart the kernel to use updated packages.


In [29]:
os.environ["OPENAI_API_KEY"] = "hidden"
os.environ["ANTHROPIC_API_KEY"] = "hidden"
os.environ["GOOGLE_API_KEY"] = "hidden"

In [28]:
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
ANTHROPIC_API_KEY = os.environ.get("ANTHROPIC_API_KEY")
GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")

In [8]:
system_prompt = """
You have vast knowledge of python coding and libraries.
You know the PEP257 docstring conventions.
You can evaluate the quality of a docstring when given a code snippet and a corresponding docstring.
"""
my_prompt = """
Given the following code: {code}
and the following generated docstring: {generated_docstring}
Assess the quality of the generated docstring based on the following criteria:
For each evaluation metric, rate the quality of the generated docstring on a scale of 0 to 100.
The evaluation metrics are:
1. Accuracy - How well does the docstring describe the code?
2. Completeness - Does the docstring contain all the necessary information about the code?
3. Relevance - The ability of the docstring to stick to the point and not include irrelevant information.
4. Understandability - How easy is it for a reader to understand the docstring?
5. Readability - How well is the docstring formatted and structured?
Your response shoulde be a python list of evaluation metrics [<Accuracy score>, <Completeness score>, <Relevance score>, <Understandability score>, <Readability score>].
"""

In [38]:
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", my_prompt),
])

llm_openai = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5")

output_parser = StrOutputParser()

chain_solution_openai = prompt | llm_openai | output_parser

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [42]:
def evaluate_docstring(code, generated_docstring, chain_solution):
    # invoke the chain and return the evaluation list
    evaluation_list = chain_solution.invoke({"code": code, "generated_docstring": generated_docstring})
    return evaluation_list

In [ ]:
data = pd.read_csv("function 10 all 3 models.csv")

tqdm.pandas()

with get_openai_callback() as cb:
    # Define the new columns to be added to the DataFrame
    columns_to_add = ['Accuracy', 'Completeness', 'Relevance', 'Understandability', 'Readability']
    
    # Initialize the new columns with None values
    data[columns_to_add] = None
    
    # Create a mask for rows where any of the new columns have missing values
    missing_values_mask = data[columns_to_add].isna()
    print(f"Evaluating docstrings and filling columns {columns_to_add}...")
    
    # Apply the function and update the DataFrame
    data.loc[missing_values_mask, columns_to_add] = data.loc[missing_values_mask].progress_apply(
        lambda row: pd.Series(evaluate_docstring(row['Function'], row['GeneratedDocstring'])),
        axis=1
    )
    
    # Debugging: Print the updated rows
    print(data.loc[missing_values_mask, columns_to_add])

print(cb)

In [55]:
prompt_for_gemini = ChatPromptTemplate.from_template(
    system_prompt + my_prompt
)

prompt_for_gemini = ChatPromptTemplate.from_template("tell me a joke about {topic}")
print(prompt_for_gemini)

data = pd.read_csv("function 10 all 3 models.csv")
function_10_code = data.loc[10, 'Function']
gemini_docstring_function_10 = data.loc[10, 'Gemini']
#print(function_10_code)
#print(gemini_docstring_function_10)
from langchain_google_genai import ChatGoogleGenerativeAI
llm_google = ChatGoogleGenerativeAI(model="gemini-1.0-pro")
chain_solution_gemini = prompt_for_gemini | llm_google | output_parser

#generated_evaluation_list = evaluate_docstring(function_10_code, gemini_docstring_function_10, chain_solution_gemini)
#print(generated_evaluation_list)
gemini_response = chain_solution_gemini.invoke({"topic": "python"})
print(gemini_response)

input_variables=['topic'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], template='tell me a joke about {topic}'))]


ChatGoogleGenerativeAIError: Invalid argument provided to Gemini: 400 API key not valid. Please pass a valid API key. [reason: "API_KEY_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
]

In [ ]:
columns_to_add_gemini = ['Accuracy Gemini', 'Completeness Gemini', 'Relevance Gemini', 'Understandability Gemini', 'Readability Gemini']
data.loc[10, columns_to_add_gemini] = generated_evaluation_list
data.to_csv("gpt 4 eval for function 10.csv", index=False)

In [36]:
columns_to_add = ['Accuracy', 'Completeness', 'Relevance', 'Understandability', 'Readability']
df = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]})
print(df)
df.loc[3] = [7, 8]
print(df)
df.loc[0, columns_to_add] = [10, 20, 30, 40, 50]
print(df)

   A  B
0  1  4
1  2  5
2  3  6
   A  B
0  1  4
1  2  5
2  3  6
3  7  8
   A  B  Accuracy  Completeness  Relevance  Understandability  Readability
0  1  4      10.0          20.0       30.0               40.0         50.0
1  2  5       NaN           NaN        NaN                NaN          NaN
2  3  6       NaN           NaN        NaN                NaN          NaN
3  7  8       NaN           NaN        NaN                NaN          NaN
